Загружаем документы

In [ ]:
import texts_loader
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

pd.set_option('display.max_columns', None)  # Отображение всех столбцов
pd.set_option('display.expand_frame_repr', False)  # Отображение всей таблицы без переноса строк

documents = texts_loader.get_texts_from_dir("tf_idf_demonstration")
print(documents)

Удаляем стоп-слова (предлоги, знаки препинания)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Загрузка стоп-слов и пунктуации
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('russian'))
punctuation = set(string.punctuation)

def preprocess_text(text):
    words = word_tokenize(text.lower())  # Привести к нижнему регистру и токенизировать
    filtered_words = [word for word in words if word not in stop_words and word not in punctuation]
    return " ".join(filtered_words)

preprocessed_documents = [ preprocess_text(doc) for doc in documents ]
print(preprocessed_documents)

Выполняем лемматизацию текста

In [ ]:
import spacy

# Загрузка языковой модели spaCy
nlp = spacy.load("ru_core_news_sm")

def preprocess_and_lemmatize(text):
    doc = nlp(text.lower())  # Привести к нижнему регистру и лемматизировать
    lemmatized_words = [token.lemma_ for token in doc if token.text not in punctuation and token.text not in stop_words]
    return " ".join(lemmatized_words)

preprocessed_documents = [ preprocess_and_lemmatize(doc) for doc in preprocessed_documents ]
print(preprocessed_documents)

Создаем TF-IDF матрицу

In [ ]:
from sklearn.preprocessing import normalize

# Создание объекта TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Применение TF-IDF к текстовым данным
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_documents)
# Нормализация матрицы TF-IDF по длине документа
tfidf_matrix = normalize(tfidf_matrix, norm='l2', axis=1)

# Получение списка слов и названий документов
words = tfidf_vectorizer.get_feature_names_out()
doc_names = ['Документ {}'.format(i+1) for i in range(len(preprocessed_documents))]

# Создание DataFrame для вывода на экран
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=words, index=doc_names)

# Вывод матрицы TF-IDF на экран
print(tfidf_df)

In [ ]:
tfidf_scores = tfidf_matrix.toarray()[0]

# Сортировка слов по значениям TF-IDF
threshold = 0.3
sorted_keywords = [word for score, word in sorted(zip(tfidf_scores, words), reverse=True) if score > threshold]

print("Исходный текст : " + documents[0])
print("Ключевые слова:", sorted_keywords)